In [22]:
import torch
from transformers import TFAutoModelForSequenceClassification
from transformers import BertTokenizerFast
import datasets

In [23]:
import numpy as np
import pandas as pd
import evaluate

Загружаем токенайзер и объект модели предобученной модели rubert

In [24]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = TFAutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Карта преобразования меток: метки в модели отличаются от наших данных

In [25]:
LABELS = {
    0: 'NEUTRAL',
    1: 'POSITIVE',
    2: 'NEGATIVE',
}

TABIT_LABELS = {
    0: 'BAD',
    1: 'NEUTRAL',
    2: 'GOOD'
}

ADJUST_MAP = {
    0: 2,
    1: 0,
    2: 1,
}

def adjust(label):
  return ADJUST_MAP[label]

Загружаем данные, собранные на tabiturient.ru

In [102]:
reviews = pd.read_json('/content/drive/MyDrive/Data/tabiturient/all-reviews.jsonl', lines=True)

In [103]:
review_texts=reviews['content'].to_list()

In [104]:
review_labels=reviews['label'].map(adjust).to_list()

Векторизуем текст

In [106]:
inputs = tokenizer(review_texts, max_length=512, padding=True, truncation=True, return_tensors='tf')

Вычисляем предсказания модели

In [107]:
outputs=model.predict(inputs)

114/114 [==============================] - 144s 1s/step


Применяем активацию softmax

In [108]:
layer=keras.layers.Activation('softmax')

In [110]:
 predicted_lables = layer(outputs.logits)

Сравниваем предсказанные и эталонные оценки метрикой CategoricalAccuracy

In [111]:
metric = keras.metrics.CategoricalAccuracy()

In [112]:
review_labels_list_cat = keras.utils.to_categorical(review_labels)

In [113]:
metric.update_state(predicted_lables, review_labels_list_cat)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=3638.0>

In [122]:
print(f"Accuracy={metric.result().numpy()}" )

Accuracy=0.6646509170532227
